# SQL for Data Science

This notebook contains a bunch of SQL exercises from various classes I took. 
I will start by setting up SQL database using sqlite. Then I will add pollution data in the notebook, and practice query data from the database.  

Topics:

    * How to setup an SQLite server and populate tables
    * Insert, delete colums from the tables
    * Query the table and output as data frame
    * Aggregation
    * Create index
    * Join

In [1]:
#import package 
%matplotlib inline

import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time

from sqlite3 import dbapi2 as sq3
import os

In [2]:
# initialize database in another drive 
db = sq3.connect(os.path.join("E:/sql/", "pm25.db"))

In [3]:
# list all tables in the database
c = db.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[('pm25',), ('region',), ('fire',), ('weather',)]

# Setup SQLite

In [16]:
# use pandas to read in the data 
#load pm2.5 pollution data and province data 
dfpm25_all = pd.read_csv('C:/Users/Benny/Documents/Fern/BKK-AQI/data/model/pm25_all.csv')
dfregion = pd.read_csv('C:/Users/Benny/Documents/Fern/BKK-AQI/data/model/region_info.csv',sep=';')

In [38]:
dfpm25_all = dfpm25_all.rename({'PM2.5':'PM25'},axis=1).reset_index()

In [39]:
dfpm25_all.head()

,index,PM25,Time,city
0,0,62.9,2016-03-03 15:00:00,Bangkok
1,1,62.9,2016-03-03 16:00:00,Bangkok
2,2,55.5,2016-03-03 17:00:00,Bangkok
3,3,55.5,2016-03-03 18:00:00,Bangkok
4,4,47.9,2016-03-03 19:00:00,Bangkok


In [18]:
dfregion.head()

,City,Region,Population,Latitude,Longitude,x_mer,y_mer,dis2BKK
0,Bangkok,Bangkok,5104476,13.7540,100.5014,1.118776e+07,1.544968e+06,0.000000
1,Bang Kruai,Nonthaburi,78305,13.8050,100.4728,1.118458e+07,1.550779e+06,6625.673838
2,Ban Lam Luk Ka,Pathum Thani,60700,13.9774,100.7778,1.121853e+07,1.570430e+06,39938.064467
3,Ban Mai,Songkhla,86899,7.2041,100.5451,1.119263e+07,8.076416e+05,737342.351919
4,Ban Pong,Ratchaburi,57559,13.8163,99.8774,1.111830e+07,1.552066e+06,69825.094995


## Inifitalize the Database and Populate Data

In [2]:
from sqlite3 import dbapi2 as sq3
import os

In [40]:
# create schema
schema ="""
DROP TABLE IF EXISTS "pm25";
CREATE TABLE "pm25" (
    "index" INT PRIMARY KEY NOT NULL,
    "Time" VARCHAR,
    "PM25" FLOAT,
    "city" VARCHAR
);
"""

In [6]:
# initialize database in another drive 
db = sq3.connect(os.path.join("E:/sql/", "pm25.db"))

In [41]:
#execute schema to generate a table 
db.cursor().executescript(schema)
db.commit()

populate pm2.5 pollution data using pandas

In [42]:
dfpm25_all.to_sql("pm25", db, if_exists="append", index=False)

populate region data using pandas

In [43]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 8 columns):
City          56 non-null object
Region        56 non-null object
Population    56 non-null int64
Latitude      56 non-null float64
Longitude     56 non-null float64
x_mer         56 non-null float64
y_mer         56 non-null float64
dis2BKK       56 non-null float64
dtypes: float64(5), int64(1), object(2)
memory usage: 3.6+ KB


In [48]:
# create schema
schema ="""
DROP TABLE IF EXISTS "region";
CREATE TABLE "region" (
    "City" VARCHAR PRIMARY KEY NOT NULL,
    "Region" VARCHAR,
    "Population" INT,
    "Latitude" FLOAT,
    "Longitude" FLOAT,
    "x_mer" FLOAT,
    "y_mer" FLOAT,
    "dis2BKK" FLOAT
);
"""

In [49]:
#execute schema to generate a table 
db.cursor().executescript(schema)
db.commit()

In [50]:
#populate schema 
region.to_sql("region", db, if_exists="append", index=False)

To make things more interesting, I try to bulk import the csv files. The fires are fire map information from 2016 - 2019. The total files are too large to load with pandas. 

In [84]:
# drop previous fire table
db.cursor().executescript("DROP TABLE IF EXISTS fire")
db.commit()

In [85]:
# create schema
schema ="""
DROP TABLE IF EXISTS "fire";
CREATE TABLE "fire" (
    "latitude" FLOAT,
    "longitude" FLOAT,
    "brightness" FLOAT,
    "acq_date" VARCHAR,
    "acq_time" INT,
    "satellite" VARCHAR,
    "instrument" VARCHAR,
    "confidence" INT,
    "bright" FLOAT,
    "power" FLOAT,
    "daynight" FLOAT
);
"""

In [86]:
#execute schema to generate a table 
db.cursor().executescript(schema)

In [23]:
import csv
from glob import glob

In [87]:
files = glob('E:/Users/Fern/BKK-AQI/data/fire_map_world/*csv')

In [76]:
# check the top of the file
with open(files[3]) as file:
    print(files[3])
    line = file.readline()
    for line_no, line in enumerate(lines):
        if line_no == 0:
            print(line)
            break

E:/Users/Fern/BKK-AQI/data/fire_map_world\fire_archive_V1_46791.csv
latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type



In [94]:
for file in files:
    print(file)
    df = pd.read_csv(file,usecols=[0,1,2,5,6,7,8,9,12,13],skiprows=1,header=None)
    print(len(df.columns))
    #rename columns 
    df.columns = ['latitude', 'longitude','brightness',
                  'acq_date','acq_time','satellite' ,'instrument',
                  'confidence','power','daynight']
    print(df.columns)
    df.to_sql('fire',db, if_exists='append',index=False)

E:/Users/Fern/BKK-AQI/data/fire_map_world\fire_archive_M6_46790.csv
10
Index(['latitude', 'longitude', 'brightness', 'acq_date', 'acq_time',
       'satellite', 'instrument', 'confidence', 'power', 'daynight'],
      dtype='object')
E:/Users/Fern/BKK-AQI/data/fire_map_world\fire_archive_M6_46792.csv
10
Index(['latitude', 'longitude', 'brightness', 'acq_date', 'acq_time',
       'satellite', 'instrument', 'confidence', 'power', 'daynight'],
      dtype='object')
E:/Users/Fern/BKK-AQI/data/fire_map_world\fire_archive_M6_46794.csv
10
Index(['latitude', 'longitude', 'brightness', 'acq_date', 'acq_time',
       'satellite', 'instrument', 'confidence', 'power', 'daynight'],
      dtype='object')
E:/Users/Fern/BKK-AQI/data/fire_map_world\fire_archive_V1_46791.csv
10
Index(['latitude', 'longitude', 'brightness', 'acq_date', 'acq_time',
       'satellite', 'instrument', 'confidence', 'power', 'daynight'],
      dtype='object')
E:/Users/Fern/BKK-AQI/data/fire_map_world\fire_archive_V1_46793.csv


In [93]:
df.head()

,0,1,2,5,6,7,8,9,12,13
0,-18.2242,144.9556,327.1,2018-01-01,15,Terra,MODIS,45,20.0,D
1,-18.0564,144.6842,327.2,2018-01-01,15,Terra,MODIS,55,32.2,D
2,-18.0529,144.6651,340.4,2018-01-01,15,Terra,MODIS,88,83.9,D
3,-18.0558,144.6914,324.4,2018-01-01,15,Terra,MODIS,42,22.8,D
4,-18.0524,144.6723,343.9,2018-01-01,15,Terra,MODIS,90,102.1,D


Do the same thing with weather data

In [21]:
# create schema for adding traffic index
schema ="""
DROP TABLE IF EXISTS "weather";
CREATE TABLE "weather" (
    "Time" VARCHAR,
    "Temperature(F)" INT,
    "Dew Point(F)" INT,
    "Humidity(%)" INT,
    "Wind" VARCHAR,
    "Wind Speed(mph)" INT,
    "Wind Gust(mph)" INT,
    "Pressure(in)" FLOAT,
    "Precip.(in)" FLOAT,
    "Precip Accum(in)" FLOAT,
    "Condition" VARCHAR,
    "date" VARCHAR PRIMARY KEY NOT NULL
    
);"""

#execute schema to generate a table 
db.cursor().executescript(schema)

In [28]:
files = glob('C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather/*csv')

In [29]:
for file in files:
    print(file)
    df = pd.read_csv(file).drop('Unnamed: 0',axis=1)
    print(len(df.columns))
    print(df.columns)
    df.to_sql('weather',db, if_exists='append',index=False)

C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather\weather_2016.csv
12
Index(['Time', 'Temperature(F)', 'Dew Point(F)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Precip Accum(in)', 'Condition', 'date'],
      dtype='object')
C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather\weather_2016v1.csv
12
Index(['Time', 'Temperature(F)', 'Dew Point(F)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Precip Accum(in)', 'Condition', 'date'],
      dtype='object')
C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather\weather_2016v2.csv
12
Index(['Time', 'Temperature(F)', 'Dew Point(F)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Precip Accum(in)', 'Condition', 'date'],
      dtype='object')
C:/Users/Benny/Documents/Fern/BKK-AQI/data/weather\weather_2017.csv
12
Index(['Time', 'Temperature(F)', 'Dew Point(F)', 'Humidity

In [31]:
# create schema for adding traffic index
sel ="""
SELECT * from weather"""

#execute schema to generate a table 
c = db.cursor().execute(sel)
c.fetchone()

('12:00 AM',
 77,
 66,
 69,
 'NE',
 3,
 0,
 30.0,
 0.0,
 0.0,
 'Fair',
 '2016-01-01 00:00:00')

## Drop a column
SQLite does not support drop a column command. If I want to drop the sattellite and instrument column in the fire table, I must create a temp table and copy the row down one by one. Then drop the temp table.

In [147]:
# rename the fire table to temp
alt = '''ALTER TABLE 'fire' RENAME TO 'temp_fire';'''

In [149]:
# make sure the temp_fire table exist
c = db.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[('pm25',), ('region',), ('temp_fire',), ('fire',)]

In [152]:
# create another fire table
alt =  """ 
DROP TABLE IF EXISTS "fire";
CREATE TABLE "fire" (
    "latitude" FLOAT,
    "longitude" FLOAT,
    "brightness" FLOAT,
    "acq_date" VARCHAR,
    "acq_time" INT,
    "confidence" INT,
    "bright" FLOAT,
    "power" FLOAT,
    "daynight" FLOAT
);
"""

db.cursor().executescript(alt)

In [153]:
db.commit()

In [154]:
# copy some column from temp_fire to fire
alt = """  
INSERT INTO fire 
SELECT
latitude, 
    longitude ,
    brightness ,
    acq_date ,
    acq_time ,
    confidence ,
    bright ,
    power ,
    daynight 
FROM
 temp_fire; """


db.cursor().executescript(alt)

In [155]:
# drop temp_fire table
db.cursor().executescript("DROP TABLE IF EXISTS temp_fire")
db.commit()

## Add a column into a table 

After dropping a column from the fire table, I want to add another column named 'instrument' with the value equal to "VIIRS" if the confident is 'n', else use "MODIS".

In [198]:
alt= " ALTER TABLE fire ADD COLUMN instrument;"
db.cursor().executescript(alt)

In [4]:
sel = " SELECT * FROM fire"
c = db.cursor().execute(sel)
c.fetchone()

(-18.2242, 144.9556, 327.1, '2018-01-01', 15, 45, None, 20.0, 'D', None)

In [5]:
c.description

(('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('brightness', None, None, None, None, None, None),
 ('acq_date', None, None, None, None, None, None),
 ('acq_time', None, None, None, None, None, None),
 ('confidence', None, None, None, None, None, None),
 ('bright', None, None, None, None, None, None),
 ('power', None, None, None, None, None, None),
 ('daynight', None, None, None, None, None, None),
 ('instrument', None, None, None, None, None, None))

In [ ]:
# populate the 'instrument' column using the 'confidence' column
alt2 = """
UPDATE fire 
SET instrument = CASE 
                WHEN confidence = 'n' THEN 'VIIRS' 
                                      ELSE 'MODIS'  
                 END;
"""
db.cursor().executescript(alt2)

In [ ]:
sel = " SELECT * FROM fire"
c = db.cursor().execute(sel)
c.fetchmany(5)

In the region column, I want to add another column named 'close2BKK'. If the province is less than 100 km from bkk (in the dis3bkk column). The province is close to bkk, assign 'yes'. Else assigned 'no'

In [3]:
alt= " ALTER TABLE region ADD COLUMN close2BKK;"
db.cursor().executescript(alt)

In [4]:
sel = " SELECT * FROM region"
c = db.cursor().execute(sel)
c.fetchone()

('Bangkok',
 'Bangkok',
 5104476,
 13.754000000000001,
 100.5014,
 11187764.6720111,
 1544967.92056433,
 0.0,
 None)

In [5]:
c.description

(('City', None, None, None, None, None, None),
 ('Region', None, None, None, None, None, None),
 ('Population', None, None, None, None, None, None),
 ('Latitude', None, None, None, None, None, None),
 ('Longitude', None, None, None, None, None, None),
 ('x_mer', None, None, None, None, None, None),
 ('y_mer', None, None, None, None, None, None),
 ('dis2BKK', None, None, None, None, None, None),
 ('close2BKK', None, None, None, None, None, None))

In [40]:
# populate the 'close2BKK' column with the case condition
alt2 = """
UPDATE region 
SET close2BKK = CASE 
                WHEN cast(dis2BKK as Integer) < 100000 THEN 'yes' 
                                      ELSE 'no'  
                 END;
"""
db.cursor().executescript(alt2)

In [41]:
sel = " SELECT * FROM region WHERE close2BKK ='no' "
c = db.cursor().execute(sel)
c.fetchmany(5)

[('Ban Mai',
  'Songkhla',
  86899,
  7.2041,
  100.5451,
  11192629.333758801,
  807641.616273147,
  737342.351919116,
  'no'),
 ('Ban Talat Yai',
  'Phuket',
  52192,
  7.8848,
  98.4001,
  10953849.0260072,
  883590.3230631059,
  701524.665227042,
  'no'),
 ('Chaiyaphum',
  'Chaiyaphum',
  58350,
  15.8105,
  102.0288,
  11357794.0622488,
  1780352.94979303,
  290372.35668982,
  'no'),
 ('Chanthaburi',
  'Chanthaburi',
  99819,
  12.6096,
  102.1045,
  11366220.9477018,
  1414911.41573794,
  220819.692919477,
  'no'),
 ('Chiang Mai',
  'Chiang Mai',
  200952,
  18.7904,
  98.9847,
  11018926.400324902,
  2125870.66719383,
  604941.619520165,
  'no')]

In [43]:
db.commit()

In [156]:
# check tables in the database
c = db.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[('pm25',), ('region',), ('fire',)]

now we have three tables:
pm25 table contains pm25, time and city
region table contains city, region, locations
fire table contains location and time of the fire 

# Inspect Tables and Filter

WHERE key words allow you to filter based on both text and numeric values in a table. There are a few different comparison operators you can use:

- = equal
- <> not equal
- < less than
- '>' greater than
- <= less than or equal to
- '>=' greater than or equal to

You can also use AND and OR to combine the logic operation.


In [95]:
sel = """
SELECT * from pm25
WHERE city == "Bangkok"
"""

c=db.cursor().execute(sel)

In [96]:
c.fetchmany(5)

[(0, '2016-03-03 15:00:00', 62.9, 'Bangkok'),
 (1, '2016-03-03 16:00:00', 62.9, 'Bangkok'),
 (2, '2016-03-03 17:00:00', 55.5, 'Bangkok'),
 (3, '2016-03-03 18:00:00', 55.5, 'Bangkok'),
 (4, '2016-03-03 19:00:00', 47.9, 'Bangkok')]

In [7]:
sel = """
SELECT City, Region from region
where dis2BKK < 10000
"""

c = db.cursor().execute(sel)
c.fetchmany(5)

[('Bangkok', 'Bangkok'), ('Bang Kruai', 'Nonthaburi')]

In [108]:
# obtain column names 
c.description

(('City', None, None, None, None, None, None),
 ('Region', None, None, None, None, None, None),
 ('Population', None, None, None, None, None, None),
 ('Latitude', None, None, None, None, None, None),
 ('Longitude', None, None, None, None, None, None),
 ('x_mer', None, None, None, None, None, None),
 ('y_mer', None, None, None, None, None, None),
 ('dis2BKK', None, None, None, None, None, None))

In [10]:
sel = """
SELECT * from pm25
WHERE city = "Bangkok"
AND PM25 > 35.5
"""

c=db.cursor().execute(sel)
c.fetchmany(5)

[(0, '2016-03-03 15:00:00', 62.9, 'Bangkok'),
 (1, '2016-03-03 16:00:00', 62.9, 'Bangkok'),
 (2, '2016-03-03 17:00:00', 55.5, 'Bangkok'),
 (3, '2016-03-03 18:00:00', 55.5, 'Bangkok'),
 (4, '2016-03-03 19:00:00', 47.9, 'Bangkok')]

In [11]:
c.description

(('index', None, None, None, None, None, None),
 ('Time', None, None, None, None, None, None),
 ('PM25', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None))

I want to test if I can query data between 2016 - 2017. To do that I need a separate table for year, then use BETWEEN to query the time between two years. 

In [12]:
alt= " ALTER TABLE pm25 ADD COLUMN year INTEGER;"
db.cursor().executescript(alt)

In [14]:
sel = " SELECT * FROM pm25"
c = db.cursor().execute(sel)
c.description

(('index', None, None, None, None, None, None),
 ('Time', None, None, None, None, None, None),
 ('PM25', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('year', None, None, None, None, None, None))

In [17]:
# populate the 'year' column with the year 
alt2 = """
UPDATE pm25 
SET year = substr(Time,0,5)
"""
db.cursor().executescript(alt2)

In [18]:
sel = " SELECT * FROM pm25"
c = db.cursor().execute(sel)
c.fetchone()

(0, '2016-03-03 15:00:00', 62.9, 'Bangkok', 2016)

In [20]:
sel = """
SELECT * from pm25
WHERE city = "Bangkok"
AND PM25 > 35.5
AND year
BETWEEN 2016 AND 2017
"""

c=db.cursor().execute(sel)
c.fetchmany(5)

[(0, '2016-03-03 15:00:00', 62.9, 'Bangkok', 2016),
 (1, '2016-03-03 16:00:00', 62.9, 'Bangkok', 2016),
 (2, '2016-03-03 17:00:00', 55.5, 'Bangkok', 2016),
 (3, '2016-03-03 18:00:00', 55.5, 'Bangkok', 2016),
 (4, '2016-03-03 19:00:00', 47.9, 'Bangkok', 2016)]

# Delete data

delete 'Ubon Ratchathani' region from the region table  


In [14]:
sel = """
SELECT * from region
WHERE region LIKE "%Ubon%"
"""
c=db.cursor().execute(sel)
c.fetchall()

[('Ubon Ratchathani',
  ' Ubon Ratchathani',
  122533,
  15.2384,
  104.8487,
  11671703.894336699,
  1714640.38699777,
  512821.52526070294,
  'no')]

In [15]:
drow="DELETE FROM region WHERE region LIKE '%Ubon%' "
db.cursor().execute(drow)

In [16]:
# Ubon region is gone
sel = """
SELECT * from region
WHERE region LIKE "%Ubon%"
"""
c=db.cursor().execute(sel)
c.fetchall()

[]

## Turn the query result into a dataframe

In [165]:
sel = """
SELECT *  from fire
where confidence > 80
"""

c = db.cursor().execute(sel)
qresults = c.fetchmany(5)

In [167]:
qresults

[(-18.0529, 144.6651, 340.4, '2018-01-01', 15, 88, None, 83.9, 'D'),
 (-18.0524, 144.6723, 343.9, '2018-01-01', 15, 90, None, 102.1, 'D'),
 (-18.0489, 144.653, 336.1, '2018-01-01', 15, 84, None, 62.6, 'D'),
 (-15.7994, 143.8199, 331.1, '2018-01-01', 15, 81, None, 79.0, 'D'),
 (-15.809000000000001, 143.8241, 332.5, '2018-01-01', 15, 83, None, 84.2, 'D')]

In [162]:
c.description

(('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('brightness', None, None, None, None, None, None),
 ('acq_date', None, None, None, None, None, None),
 ('acq_time', None, None, None, None, None, None),
 ('confidence', None, None, None, None, None, None),
 ('bright', None, None, None, None, None, None),
 ('power', None, None, None, None, None, None),
 ('daynight', None, None, None, None, None, None))

In [163]:
cols = [e[0] for e in c.description]

In [164]:
cols

['latitude',
 'longitude',
 'brightness',
 'acq_date',
 'acq_time',
 'confidence',
 'bright',
 'power',
 'daynight']

In [180]:
df = pd.DataFrame(qresults,columns=cols)
df.head()

,latitude,longitude,brightness,acq_date,acq_time,confidence,bright,power,daynight
0,-18.0529,144.6651,340.4,2018-01-01,15,88,None,83.9,D
1,-18.0524,144.6723,343.9,2018-01-01,15,90,None,102.1,D
2,-18.0489,144.6530,336.1,2018-01-01,15,84,None,62.6,D
3,-15.7994,143.8199,331.1,2018-01-01,15,81,None,79.0,D
4,-15.8090,143.8241,332.5,2018-01-01,15,83,None,84.2,D


Turn query result into a dataframe (concise version).

In [181]:
sel = """
SELECT * from pm25
WHERE city == "Bangkok"
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults,columns= [e[0] for e in c.description])

In [182]:
df.head()

,index,Time,PM25,city
0,0,2016-03-03 15:00:00,62.9,Bangkok
1,1,2016-03-03 16:00:00,62.9,Bangkok
2,2,2016-03-03 17:00:00,55.5,Bangkok
3,3,2016-03-03 18:00:00,55.5,Bangkok
4,4,2016-03-03 19:00:00,47.9,Bangkok


# Query results

- using WHERE, AND, OR 
- using IS NOT NULL and IS NULL
- LIKE and NOT LIKE

In [4]:
sel = """
SELECT * from weather
WHERE wind IS NULL
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults,columns= [e[0] for e in c.description])
df.head(3)

,Time,Temperature(F),Dew Point(F),Humidity(%),Wind,Wind Speed(mph),Wind Gust(mph),Pressure(in),Precip.(in),Precip Accum(in),Condition,date
0,2:00 PM,90,75,62,None,0,0,29.8,0.0,0.0,Fair,2016-03-18 14:00:00
1,2:05 PM,90,75,62,None,0,0,29.8,0.0,0.0,Fair,2016-03-18 14:05:00
2,6:30 PM,0,0,0,None,0,0,0.0,0.0,0.0,Fair,2016-05-06 18:30:00


In [5]:
sel = """
SELECT count(*) from weather
WHERE wind IS NOT NULL
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults,columns= [e[0] for e in c.description])
df.head(3)

,count(*)
0,56181


LIKE and NOT LIKE is used to search for pattern.

The % wildcard will match zero, one, or many characters in text. For example, the following query matches companies like 'Data', 'DataC' 'DataCamp', 'DataMind', and so on: 

SELECT name
FROM companies
WHERE name LIKE 'Data%';

The _ wildcard will match a single character. For example, the following query matches companies like 'DataCamp', 'DataComp', and so on:

SELECT name
FROM companies
WHERE name LIKE 'DataC_mp';

In [6]:
# find the day when the wind blow north
sel = """
SELECT * from weather
WHERE wind LIKE 'N%'
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults,columns= [e[0] for e in c.description])
df.head(3)

,Time,Temperature(F),Dew Point(F),Humidity(%),Wind,Wind Speed(mph),Wind Gust(mph),Pressure(in),Precip.(in),Precip Accum(in),Condition,date
0,12:00 AM,77,66,69,NE,3,0,30.0,0.0,0.0,Fair,2016-01-01 00:00:00
1,12:30 AM,77,64,65,NNE,5,0,30.0,0.0,0.0,Fair,2016-01-01 00:30:00
2,1:30 AM,75,64,69,NE,5,0,30.0,0.0,0.0,Fair,2016-01-01 01:30:00


In [7]:
# cound the day when the wind does not blow north
sel = """
SELECT COUNT(*) from weather
WHERE wind NOT LIKE 'N%'
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults,columns= [e[0] for e in c.description])
df.head(3)

,COUNT(*)
0,44849


# Aggegation 

After aggregation, there will be additional columns to display; therefore need to add another columns in pandas.

Aggregation commands: MAX, COUNT, SUM, AVG

Find the province with the maximum pollution level

In addition to using aggregate functions, you can perform basic arithmetic with symbols like +, -, *, and /.

So, for example, this gives a result of 12:

SELECT (4 * 3)

In [194]:
# find the province with the maximum pollution level
sel = """
SELECT *, MAX(pm25) AS maxpm
from pm25
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults,columns=[e[0] for e in c.description])
df.head()

,index,Time,PM25,city,maxpm
0,421096,2017-11-15 10:00:00,787.9,Khon Kaen,787.9


In the region table, count the number of city in a region using GROUP BY, and only show 3 rows using LIMIT.

In [17]:
sel = """
SELECT COUNT(CITY) AS count, region
from region
GROUP BY region
LIMIT 3
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults,columns=['num_city','region'])
df.head()

,num_city,region
0,1,Udon Thani
1,1,Bangkok
2,1,Chaiyaphum


# AS for aliasing

Turn the temperature to degree C as Temperature(C)

In [10]:
sel = """
SELECT  *
from weather
LIMIT 3
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,Time,Temperature(F),Dew Point(F),Humidity(%),Wind,Wind Speed(mph),Wind Gust(mph),Pressure(in),Precip.(in),Precip Accum(in),Condition,date
0,12:00 AM,77,66,69,NE,3,0,30.0,0.0,0.0,Fair,2016-01-01 00:00:00
1,12:30 AM,77,64,65,NNE,5,0,30.0,0.0,0.0,Fair,2016-01-01 00:30:00
2,1:00 AM,77,66,69,E,5,0,30.0,0.0,0.0,Fair,2016-01-01 01:00:00


In [24]:
sel = """
SELECT  (("Temperature(F)"*0.55) - 17.78) AS "Temperature(C)"
from weather
LIMIT 3
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,Temperature(C)
0,24.57
1,24.57
2,24.57


# ORDER BY 

ORDER BY is used to sort the result. 
The default will sort in ascending order

SELECT title
FROM films
ORDER BY release_year DESC;

In [29]:
sel = """
SELECT  * 
from Region
ORDER BY City
LIMIT 3
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,City,Region,Population,Latitude,Longitude,x_mer,y_mer,dis2BKK,close2BKK
0,Ban Lam Luk Ka,Pathum Thani,60700,13.9774,100.7778,1.121853e+07,1.570430e+06,39938.064467,yes
1,Ban Mai,Songkhla,86899,7.2041,100.5451,1.119263e+07,8.076416e+05,737342.351919,no
2,Ban Pong,Ratchaburi,57559,13.8163,99.8774,1.111830e+07,1.552066e+06,69825.094995,yes


In [31]:
sel = """
SELECT  * 
from Region
ORDER BY dis2BKK DESC
LIMIT 3
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,City,Region,Population,Latitude,Longitude,x_mer,y_mer,dis2BKK,close2BKK
0,Yala,Yala,93558,6.5400,101.2813,1.127458e+07,733657.663071,815942.223042,no
1,Hat Yai,Songkhla,191696,7.0084,100.4767,1.118502e+07,785828.605703,759144.294334,no
2,Songkhla,Songkhla,84264,7.1988,100.5951,1.119820e+07,807050.744514,737990.892141,no


In [33]:
sel = """
SELECT  * 
from Region
WHERE close2BKK = "yes"
ORDER BY dis2BKK DESC
LIMIT 3
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,City,Region,Population,Latitude,Longitude,x_mer,y_mer,dis2BKK,close2BKK
0,Suphan Buri,Suphan Buri,53399,14.4742,100.1222,1.114555e+07,1.627145e+06,92384.816519,yes
1,Tha Maka,Kanchanaburi,52907,13.9000,99.7667,1.110598e+07,1.561606e+06,83461.586248,yes
2,Si Racha,Chon Buri,178916,13.1737,100.9311,1.123560e+07,1.478942e+06,81532.548964,yes


# GROUP BY 

SELECT sex, count(*)

FROM employees

GROUP BY sex;

Note that GROUP BY always goes after the FROM clause!

To filter based on aggregation, use HAVING.  

In [38]:
# count the number of provinces group by weather it is closed to BKK 

sel = """
SELECT  close2BKK, COUNT(*) 
from Region
GROUP BY close2BKK 
LIMIT 3
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,close2BKK,COUNT(*)
0,no,37
1,yes,18


In [40]:
# count the number of provinces group by weather it is closed to BKK 

sel = """
SELECT  wind, "Temperature(F)", COUNT(*) 
from weather
GROUP BY wind 
ORDER BY "Temperature(F)"
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,Wind,Temperature(F),COUNT(*)
0,ENE,75,2220
1,CALM,77,4472
2,E,77,3309
3,NE,77,1836
4,NNE,77,2477


In [44]:
# count the number of provinces group by weather it is closed to BKK 

sel = """
SELECT  wind, "Temperature(F)", COUNT(*) AS count
from weather
GROUP BY wind 
HAVING count > 2000
ORDER BY count
"""
c = db.cursor().execute(sel)
qresults = c.fetchall()
df = pd.DataFrame(qresults, columns=[e[0] for e in c.description])
df.head()

,Wind,Temperature(F),count
0,ENE,75,2220
1,SW,82,2412
2,NNW,82,2439
3,NNE,77,2477
4,SSE,88,2495


# Create Index 

index helps speed the look up. All databases has it. 

In [18]:
crind="CREATE UNIQUE INDEX region_idx ON region(city,region);"
db.cursor().execute(crind)
db.commit()

In [21]:
sel = """
PRAGMA INDEX_LIST('region');
"""
c = db.cursor().execute(sel)
c.fetchall()

[(0, 'region_idx', 1, 'c', 0), (1, 'sqlite_autoindex_region_1', 1, 'pk', 0)]

# Joins

a simple subset

In [48]:
sel = """
SELECT * 
FROM region
WHERE region.close2BKK = 'yes'
"""
c = db.cursor().execute(sel)
c.fetchmany(1)

[('Bangkok',
  'Bangkok',
  5104476,
  13.754000000000001,
  100.5014,
  11187764.6720111,
  1544967.92056433,
  0.0,
  'yes')]

### Implicit join

In [50]:
sel = """
SELECT * 
FROM pm25, region
WHERE pm25.city = region.City
AND region.close2BKK = 'yes'
"""
c = db.cursor().execute(sel)
pd.DataFrame(c.fetchall()).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,2016-03-03 15:00:00,62.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
1,1,2016-03-03 16:00:00,62.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
2,2,2016-03-03 17:00:00,55.5,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
3,3,2016-03-03 18:00:00,55.5,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
4,4,2016-03-03 19:00:00,47.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes


### Explicit INNER JOIN

In [51]:
sel = """
SELECT * 
FROM pm25, region
ON pm25.city = region.City
where region.close2BKK = 'yes'
"""
c = db.cursor().execute(sel)
pd.DataFrame(c.fetchall()).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,2016-03-03 15:00:00,62.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
1,1,2016-03-03 16:00:00,62.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
2,2,2016-03-03 17:00:00,55.5,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
3,3,2016-03-03 18:00:00,55.5,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
4,4,2016-03-03 19:00:00,47.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes


# Left outer join

In [53]:
sel = """
SELECT * 
FROM pm25 LEFT OUTER JOIN region
ON pm25.city = region.City
where region.close2BKK = 'yes'
"""
c = db.cursor().execute(sel)
pd.DataFrame(c.fetchall()).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,2016-03-03 15:00:00,62.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
1,1,2016-03-03 16:00:00,62.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
2,2,2016-03-03 17:00:00,55.5,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
3,3,2016-03-03 18:00:00,55.5,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes
4,4,2016-03-03 19:00:00,47.9,Bangkok,Bangkok,Bangkok,5104476,13.754,100.5014,1.118776e+07,1.544968e+06,0.0,yes


Note that full outer join and right join are not supposed by sqlite

# Pandas interfaces with SQL

pandas can read from db with data large than the memory. It would only generate the data as request.


In [55]:
pd.read_sql("SELECT * FROM region WHERE close2BKK = 'yes';", db).head()

,City,Region,Population,Latitude,Longitude,x_mer,y_mer,dis2BKK,close2BKK
0,Bangkok,Bangkok,5104476,13.7540,100.5014,1.118776e+07,1.544968e+06,0.000000,yes
1,Bang Kruai,Nonthaburi,78305,13.8050,100.4728,1.118458e+07,1.550779e+06,6625.673838,yes
2,Ban Lam Luk Ka,Pathum Thani,60700,13.9774,100.7778,1.121853e+07,1.570430e+06,39938.064467,yes
3,Ban Pong,Ratchaburi,57559,13.8163,99.8774,1.111830e+07,1.552066e+06,69825.094995,yes
4,Chon Buri,Chon Buri,219164,13.3622,100.9835,1.124143e+07,1.500371e+06,69778.378647,yes
